# 인구 이동

[baekjoon의 '인구 이동' 링크](https://www.acmicpc.net/problem/16234)

### 문제
N×N크기의 땅이 있고, 땅은 1×1개의 칸으로 나누어져 있다. 각각의 땅에는 나라가 하나씩 존재하며, r행 c열에 있는 나라에는 A[r][c]명이 살고 있다. 인접한 나라 사이에는 국경선이 존재한다. 모든 나라는 1×1 크기이기 때문에, 모든 국경선은 정사각형 형태이다.  
  
오늘부터 인구 이동이 시작되는 날이다.  
  
인구 이동은 다음과 같이 진행되고, 더 이상 아래 방법에 의해 인구 이동이 없을 때까지 지속된다.  
  
국경선을 공유하는 두 나라의 인구 차이가 L명 이상, R명 이하라면, 두 나라가 공유하는 국경선을 오늘 하루동안 연다.  
위의 조건에 의해 열어야하는 국경선이 모두 열렸다면, 인구 이동을 시작한다.  
국경선이 열려있어 인접한 칸만을 이용해 이동할 수 있으면, 그 나라를 오늘 하루 동안은 연합이라고 한다.  
연합을 이루고 있는 각 칸의 인구수는 (연합의 인구수) / (연합을 이루고 있는 칸의 개수)가 된다. 편의상 소수점은 버린다.  
연합을 해체하고, 모든 국경선을 닫는다.  
각 나라의 인구수가 주어졌을 때, 인구 이동이 몇 번 발생하는지 구하는 프로그램을 작성하시오.  
  
### 입력  
첫째 줄에 N, L, R이 주어진다. (1 ≤ N ≤ 50, 1 ≤ L ≤ R ≤ 100)  
  
둘째 줄부터 N개의 줄에 각 나라의 인구수가 주어진다. r행 c열에 주어지는 정수는 A[r][c]의 값이다. (1 ≤ A[r][c] ≤ 100)  
  
인구 이동이 발생하는 횟수가 2,000번 보다 작거나 같은 입력만 주어진다.  
  
### 출력  
인구 이동이 몇 번 발생하는지 첫째 줄에 출력한다.  
  

In [1]:
from queue import Queue

changed = False # 인구 이동 시도를 할 때마다 인구 이동 있었는지 플래그

def moving(population, union): # 연합을 통한 인구이동
    val_sum = 0 
    num = 0
    for i in union:
        t_y, t_x = i  
        val_sum += population[t_y][t_x] 
        num += 1 
    for t_y, t_x in union: 
        population[t_y][t_x] = val_sum // num
    # 연합된 나라에 대해서 인구이동을 해준다
                    
def cal_moving(N, L, R, population, searched, idx=0, root=False): # 인구 이동 할 수 있는 연합 형성
    global changed
    u_set = set() # 연합 집합
        
    if idx >= N*N or idx < 0:  # 모든 나라 인구 확인이 끝났을 때
        return set()
    
    if idx == 0: # 첫 idx
        root = True
    
    y = idx // N 
    x = idx % N
    # 각 인덱스에 대한 나라의 위치
    
    searched[y][x] = True # 탐색한 나라는 표시함
    u_set.add((y,x)) # 현재 탐색 중인 나라는 연합에 포함 시킨다.
    
    # L <= 국경을 맞댄 나라의 인구 차이 <= R인 조건을 만족할 때
    if y+1 < N and not searched[y+1][x]: # 아래로 연합 확장 검사
        if L <= abs(population[y][x] - population[y+1][x]) <= R:
            u_set = u_set.union(cal_moving(N, L, R, population, searched, idx+N))
    if x+1 < N and not searched[y][x+1]: # 오른쪽으로 연합 확장 검사
        if L <= abs(population[y][x] - population[y][x+1]) <= R:
            u_set = u_set.union(cal_moving(N, L, R, population, searched, idx+1))
    if y-1 >= 0 and not searched[y-1][x]: # 위로 연합 확장 검사
        if L <= abs(population[y][x] - population[y-1][x]) <= R:
            u_set = u_set.union(cal_moving(N, L, R, population, searched, idx-N))
    if x-1 >= 0 and not searched[y][x-1]: # 왼쪽으로 연합 확장 검사
        if L <= abs(population[y][x] - population[y][x-1]) <= R:
            u_set = u_set.union(cal_moving(N, L, R, population, searched, idx-1))

    if len(u_set) == 1: # 만약 확장되는 연합이 없을 때는
        for next_idx in range(idx+1, N*N):
            n_y = next_idx // N 
            n_x = next_idx % N
            if not searched[n_y][n_x]: # 아직 탐색되지 않은 지역에 대해서
                cal_moving(N, L, R, population, searched, next_idx, True) # 루트로 잡고
                break
        
    if len(u_set) > 1 and root: # 확장되는 연합이 있었고, 현재 위치가 첫 시작점이라면
        changed = True # 변화 되었음을 표시
        moving(population, u_set) # 연합 사이의 인구이동
    return u_set 


def solution(N, L, R, population):
    answer = 0
    global changed 
    changed = True
    population = [[int(x) for x in x.split()] for x in population.split('\n')]
    while changed: # 인구이동이 계속되는 동안
        changed = False
        searched = [[False for _  in range(N)] for _ in range(N)]
        cal_moving(N, L, R, population, searched) # 계속해서 연합끼리의 인구이동
        if changed:
            answer += 1 # 만약 인구 이동이 되었다면 답 1 증가
    return answer
    

In [2]:
N, L, R = 2, 20, 50
population = '''50 30
20 40'''

In [3]:
solution(N, L, R, population)

1

In [4]:
N, L, R = 2, 40, 50
population = '''50 30
20 40'''

In [5]:
solution(N, L, R, population)

0

In [6]:
N, L, R = 2, 20, 50
population = '''50 30
30 40'''

In [7]:
solution(N, L, R, population)

1

In [8]:
N, L, R = 3, 5, 10
population = '''10 15 20
20 30 25
40 22 10'''

In [9]:
solution(N, L, R, population)

2

In [10]:
N, L, R = 4, 10, 50
population = '''10 100 20 90
80 100 60 70
70 20 30 40
50 20 100 10'''

In [11]:
solution(N, L, R, population)

3